In [1]:
# =========================================
# 12) Cargar artefactos y servir inferencia (mínimo)
# =========================================
import os, json, glob
import numpy as np
import pandas as pd
import joblib

def _latest_art_dir(root="artefactos"):
    subs = [d for d in glob.glob(os.path.join(root, "*")) if os.path.isdir(d)]
    subs.sort(key=lambda p: (os.path.getmtime(p), p), reverse=True)
    return subs[0]

ART_DIR = _latest_art_dir("artefactos")
INPUT_SCHEMA = json.load(open(os.path.join(ART_DIR,"input_schema.json"), "r", encoding="utf-8"))
POLICY       = json.load(open(os.path.join(ART_DIR,"decision_policy.json"), "r", encoding="utf-8"))

WINNER  = POLICY["winner"]
PIPE    = joblib.load(os.path.join(ART_DIR, f"pipeline_{WINNER}.joblib"))

if "columns" in INPUT_SCHEMA and "dtypes" in INPUT_SCHEMA:
    FEATURES = INPUT_SCHEMA["columns"]
    DTYPES   = INPUT_SCHEMA["dtypes"]
else:
    FEATURES = list(INPUT_SCHEMA.keys())
    DTYPES   = INPUT_SCHEMA

def _coerce_and_align(df: pd.DataFrame) -> pd.DataFrame:
    for c in FEATURES:
        if c not in df.columns: df[c] = np.nan
        t = str(DTYPES.get(c, "object")).lower()
        if t.startswith(("int","float")):
            df[c] = pd.to_numeric(df[c], errors="coerce")
        else:
            df[c] = df[c].astype("string").str.strip()
    return df[FEATURES]

def predict_batch(records):
    if isinstance(records, dict):
        records = [records]
    df = _coerce_and_align(pd.DataFrame(records))
    yhat = PIPE.predict(df)
    return [{"y_pred": float(p)} for p in yhat]

# Smoke test (si hay samples)
SAMPLES = os.path.join(ART_DIR, "sample_inputs.json")
if os.path.exists(SAMPLES):
    samples = json.load(open(SAMPLES, "r", encoding="utf-8"))
    print(predict_batch(samples)[:3])
else:
    print("No hay sample_inputs.json; crea 1–3 registros para probar.")


[{'y_pred': 2617.8898524583838}, {'y_pred': 4333.100541425777}, {'y_pred': 8113.826168145051}]


# Parte Modelado para algoritmos ML de regresión
Los datos ya vienen limpios y preparados para modelado.  
Esta línea reproduce la metodología de clasificación, adaptada a un **target continuo**.

---

### Paso 1 — Cargar datos y objetivo
**Objetivo:** Leer dataset limpio, definir `TARGET` numérico (ej. `ingreso_mensual`, `precio`, `linea_credito`, etc.).  
**Imprime:** `shape`, resumen estadístico (`describe()`), verificación de nulos.  
**Salida:** `df`, `X`, `y`.  
**Exporta (opcional):** `reports/step1_overview_{ts}.csv`.

---

### Paso 2 — Split temprano (holdout 80/20)
**Objetivo:** Separar `TRAIN/TEST` con `random_state` fijo y sin estratificación.  
**Imprime:** tamaños de `X_train/X_test`.  
**Salida:** `X_train`, `X_test`, `y_train`, `y_test`.  
**Exporta (opcional):** `reports/step2_split_meta_{ts}.csv`.

---

### Paso 3 — Preprocesamiento en pipeline
**Objetivo:** Detectar `num_features`, `cat_nominal`, `cat_ordinal`;  
crear `ColumnTransformer` con `StandardScaler` (numérico) +  
`OneHotEncoder` (nominal) + `OrdinalEncoder` (ordinal).  
No usa SMOTE (solo para clasificación).  
**Salida:** `preprocessor`, `build_pipe(model)`.

---

### Paso 4 — Modelos candidatos
**Objetivo:** Definir modelos base de regresión:  
`LRN (LinearRegression)`, `RIDGE`, `LASSO`, `ENET`, `KNN`, `DTR`, `RFR`, `GBR`, `XGBR`, `LGBR`, `CATR`, `MLPRegressor`.  
**Salida:** `candidates`.

---

### Paso 5 — Baseline con CV (sin tuning)
**Objetivo:** Evaluar candidatos con `KFold(5)` dentro del pipeline (`prep→modelo`).  
**Métricas:**  
- `MAE`, `RMSE`, `MAPE`, `R2`, `EVS` (Explained Variance).  
**Ranking:** por **`RMSE` (ascendente)** y **`R2` (descendente)**.  
**Salida:** `baseline_df`, `baseline_best_name`, `baseline_best_model`.  
**Exporta:** `reports/step5_baseline_metrics_{ts}.csv`.

---

### Paso 6 — Tuning con CV y elección preliminar
**Objetivo:** `RandomizedSearchCV` con refit=`neg_root_mean_squared_error`.  
**Métricas:** mismas del paso 5.  
**Salida:** `tuning_df`, `best_name`, `final_pipe_opt`, `best_params`.  
**Exporta:** `reports/step6_tuning_metrics_{ts}.csv`.

---

### Paso 7 — Comparación justa (solo CV)
**Objetivo:** Comparar **Baseline(best)** vs **Tuned(best)** con el mismo CV.  
**Regla:** si mejora de `RMSE` < 1% → mantener baseline.  
**Salida:** `winner_name`, `winner_pipe`.  
**Exporta:** `reports/step7_fair_compare_{ts}.csv`.

---

### Paso 8 — Política de decisión (análisis de residuales)
**Objetivo:** Validar distribución de errores (bias y dispersión).  
**Incluye:** histograma y QQ-plot de residuales en TRAIN (OOF-CV).  
**Salida:** `residuals_df` con columnas (`y_true`, `y_pred`, `residual`).  
**Exporta:** `reports/step8_residuals_train_{ts}.csv`.

---

### Paso 9 — Evaluación final en TEST
**Objetivo:** Evaluar `winner_pipe` en TEST.  
**Métricas:** `MAE`, `RMSE`, `MAPE`, `R2`, `EVS`.  
**Imprime:** resumen general + gráfico `y_true vs y_pred`.  
**Salida:** `y_pred_test`, `residuals_test`.  
**Exporta:**  
- `reports/step9_test_metrics_{ts}.csv`  
- `reports/step9_residuals_test_{ts}.csv`.

---

### Paso 10 — Interpretabilidad y Error Analysis
**Objetivo:** Explicar el modelo ganador en términos de variables más influyentes.  
**Incluye:**  
- Importancias (modelos tipo árbol o `permutation_importance`).  
- SHAP values (`shap.Explainer` o nativo de CatBoost/LGB/XGB).  
- Análisis de outliers (top residuales).  
**Exporta:**  
- `reports/step10_feature_importance_{ts}.csv`  
- `reports/step10_shap_meanabs_{ts}.csv`  
- `reports/step10_residuals_outliers_{ts}.csv`.

---

### Paso 11 — Exportar artefactos (despliegue)
**Objetivo:** Empaquetar modelo, esquema y metadatos.  
**Genera:**  
- `artefactos/{ts}/pipeline_{winner}.joblib`  
- `input_schema.json`  
- `decision_policy.json` (sin umbral)  
- `model_card.md`  
- `sample_inputs.json`, `sample_outputs.json`.

---

### Paso 12 — Cargar artefactos y servir inferencia
**Objetivo:** Reutilizar artefactos para predicción batch o en servidor.  
**Salida:** `predict_batch(records)` con valores continuos y errores absolutos relativos.  
**Smoke test:** lee `sample_inputs.json` y muestra primeras predicciones.

---

### Notas finales
- Métricas en **decimales o unidades reales** según variable.  
- En regresión, el foco es minimizar `RMSE` sin sacrificar interpretabilidad.  
- SHAP y análisis de residuales ayudan a detectar sesgos sistemáticos.  
- El pipeline exportado es reutilizable para **Streamlit**, **API REST** o **Docker**.




---

## 🚀 Despliegue y uso
1. Entrenar con los 12 pasos (clasificación o regresión).  
2. Exportar artefactos (`step11`).  
3. Cargar modelo (`step12`) e integrar con:
   - **Streamlit Cloud** (dashboard interactivo),
   - **FastAPI / Flask** (servicio REST),
   - **Docker Compose** (entorno reproducible).

---

## 🧮 Reproducibilidad
- `random_state` global en todos los pasos.  
- Trazabilidad mediante `TS` (timestamp único por ejecución).  
- Exportación automática de métricas, metadatos y artefactos.

---

## ⚖️ Buenas prácticas
- Evaluar **equilibrio F1 vs interpretabilidad**.  
- Conservar datasets originales para auditoría.  
- Documentar cambios de versión (`model_card.md`).  
- No modificar `pipeline.joblib` manualmente.

---

**Autor / Adaptador:**  
Metodología elaborada para enseñanza y despliegue reproducible de modelos ML en entorno académico y profesional.  
Compatible con datasets institucionales (`*.csv`) y flujos tipo CRISP-DM.



In [2]:
# Revisar la guiap_ClasificacionesPlantillav1.1 y la guiap_ClasificacionesPlantillav1.1-Despliegue